In [1]:
import funcoes_kenzo as fk

In [7]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import lightgbm

pd.options.display.max_columns = 100
class Pessoa:
    def __init__(self, nome,idade):
        self.nome = nome
        self.idade = idade

    
    def idade_plus_5(self):
        print(self.idade + 5, self.nome)
x = Pessoa('Kenzo', 29)
#x.idade_plus_5()



In [5]:
a = 1_000000.11212
print(f'{a:_}')


1_000_000.11212


In [8]:
df = pd.DataFrame({'id':['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16'],
                   'Categoria1':['a','b','a','a','c','a','b','c','a','b','b','a','c','c','a','a','b'],
                   'idade':[80,10,70,90,20,17,16,57,67,49,5,1,10,29,76,18,15],
                   'target':[1,0,1,1,0,0,0,1,1,1,0,0,0,1,0,1,1]})

x = df[['Categoria1','idade']]
y = df[['id','target']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)
x_test.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

categoria1_data = list(x_train.Categoria1.unique())
categoria1_data.sort()
enc = OneHotEncoder(handle_unknown='ignore')

transformed = enc.fit_transform(x_train[['Categoria1']])
transformed = pd.DataFrame(transformed.toarray())
transformed.columns = categoria1_data

x_train = pd.concat([x_train, transformed],axis=1)

In [ ]:
dic = {'aaa': ('idade', 'mean'),'bbb': ('target', 'mean')}
Agrupamento = df.groupby(by=['Categoria1']).apply(fk.agrup,dic)
Agrupamento

In [55]:
x_train

,Categoria1,idade,a,b,c
0,a,67,1.0,0.0,0.0
1,c,29,0.0,0.0,1.0
2,a,70,1.0,0.0,0.0
3,b,49,0.0,1.0,0.0
4,b,15,0.0,1.0,0.0
5,c,20,0.0,0.0,1.0
6,c,57,0.0,0.0,1.0
7,b,5,0.0,1.0,0.0
8,c,10,0.0,0.0,1.0
9,a,90,1.0,0.0,0.0


In [156]:
enc = OneHotEncoder(handle_unknown='ignore')
transformed = enc.fit_transform(x_test[['Categoria1']])
transformed = pd.DataFrame(transformed.toarray())
transformed[2] = 0
transformed.columns = categoria1_data
x_test = pd.concat([x_test, transformed],axis=1)
x_test

,Categoria1,idade,a,b,c
0,a,80,1.0,0.0,0
1,b,10,0.0,1.0,0
2,a,17,1.0,0.0,0
3,a,18,1.0,0.0,0
4,a,1,1.0,0.0,0
5,a,76,1.0,0.0,0


In [6]:
models = {'LogisticRegression':LogisticRegression(random_state=42),
          'LightGBM': lightgbm.LGBMClassifier(random_state=42,class_weight='balanced')}
params = {'LogisticRegression':{'penalty':['l1','l2'], 'C':[10, 1, 0.1, 0.01]},
          'LightGBM':{'n_estimator':[16,32,64,128],'learning_rate':[0.5,0.3,0.1],'max_depth':[3,4,5]}}

In [7]:
def select_estimator(x_train,y_train, cv, scoring, models, params):
    estimators = []
    b_scores = []
    b_params = []
    b_estimator = []
    keys = models.keys()
    for key in keys:
        print(f'GridSearch for {key}')
        score, param, esti = grid_search(models[key], x_train, y_train, params[key], scoring, cv)
        estimators.append(key),
        b_scores.append(score)
        b_params.append(param)
        b_estimator.append(esti)
    return pd.DataFrame.from_dict({'estimator':estimators,'best_score':b_scores,'best_params':b_params,
                                'model':b_estimator}).sort_values(by=['best_score'],ascending=False).reset_index(drop=True)
def grid_search(clf, x_train, y_train, params, score, cv):
    grid = GridSearchCV(clf,params, scoring=score, cv=cv, return_train_score=True,n_jobs=-1,verbose=2)
    grid_fitted = grid.fit(x_train,np.ravel(y_train))
    print(f'Best Score: {grid_fitted.best_score_}')
    print(f'Best Params {grid_fitted.best_params_}')
    return grid_fitted.best_score_, grid_fitted.best_params_,grid_fitted.best_estimator_

def evaluate_auc(model, test_features, test_labels):
    predictions = model.predict(test_features)
    pred_proba = model.predict_proba(test_features)[:,1]
    accuracy = accuracy_score(test_labels, predictions)
    prop_1_test = test_labels.sum()/test_labels.shape[0]
    auc = roc_auc_score(test_labels, pred_proba)
    return auc

In [8]:
colunas_modelo = ['idade'] + categoria1_data
x_train = x_train[colunas_modelo]
x_test = x_test[colunas_modelo]

cv_kfold = KFold(4,shuffle=True)
df_res = select_estimator(x_train, y_train['target'], cv_kfold, scoring='precision', models=models,params=params)
best_clf = df_res.iloc[0][3]
best_clf

KeyError: "['b', 'a', 'c'] not in index"

In [162]:
y_test

,id,target
0,0,1
1,1,0
2,5,0
3,15,1
4,11,0
5,14,0


In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
grid_auc = evaluate_auc(best_clf, x_test, y_test['target'])
grid_auc

NameError: name 'best_clf' is not defined